# 6.3 잠재 디리클레 할당(LDA) 실습2
### 1. 실습을 통한 이해
##### 1) 뉴스 기사 제목 데이터에 대한 이해
약 15년 동안 발행되었던 뉴스 기사 제목을 모아놓은 영어 데이터를 아래 링크에서 다운받을 수 있습니다.

링크 : https://www.kaggle.com/therohk/million-headlines

In [1]:
import pandas as pd
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [2]:
print(len(data))

1082168


In [3]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


이 데이터는 publish_data와 headline_text라는 두 개의 열을 갖고 있습니다. 각각 뉴스가 나온 날짜와 뉴스 기사 제목을 의미합니다. 필요한 데이터는 이 중에서 headline_text 열. 즉, 뉴스 기사 제목이므로 이 부분만 별도로 저장합니다.

In [5]:
text = data[['headline_text']] 
text.head(5)

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


##### 2) 텍스트 전처리
이번 챕터에서는 불용어 제거, 표제어 추출, 길이가 짧은 단어 제거라는 세 가지 전처리 기법을 사용합니다.

In [6]:
import nltk
text['headline_text']= text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


NLTK의 word_tokenize를 통해 단어 토큰화를 수행합니다.

In [7]:
print(text.head(5))

                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


상위 5개의 샘플만 출력하여 단어 토큰화 결과를 확인합니다. 이제 불용어를 제거합니다.

In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x:[word for word in x if word not in (stop)])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


여기서는 NLTK가 제공하는 영어 불용어를 통해서 text 데이터로부터 불용어를 제거해보도록 하겠습니다.

In [10]:
print(text.head(5))

                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


상위 5개의 샘플에 대해서 
- 불용어를 제거하기 전과 후의 데이터만 비교해도 확실히 몇 가지 단어들이 사라진 것이 보입니다. 
- against, be, of, a, in, to 등의 단어가 제거되었습니다. 

이제 표제어 추출을 수행합니다. 
- 표제어 추출로 **3인칭 단수 표현을 1인칭으로** 바꾸고, 
- **과거 현재형 동사를 현재형**으로 바꿉니다.

In [13]:
from nltk.stem import WordNetLemmatizer

text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


표제어 추출이 된 것을 확인할 수 있습니다. 이제 길이가 3이하인 단어에 대해서 제거하는 작업을 수행합니다.

In [14]:
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word)>3])
print(tokenized_doc[:5])

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


##### 3) TF-IDF 행렬 만들기
TF-IDF 챕터에서 배운 TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다. 이를 사용하기 위해 다시 토큰화 작업을 역으로 취소하는 역토큰화(Detokenization)작업을 수행해보겠습니다.

In [28]:
detokenized_doc =[]
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
text['headline_text'] = detokenized_doc

text['headline_text'][:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0       decide community broadcast licence
1       fire witness must aware defamation
2    call infrastructure protection summit
3                   staff aust strike rise
4      strike affect australian travellers
Name: headline_text, dtype: object

정상적으로 역토큰화가 수행되었음을 확인할 수 있습니다.  
이제 사이킷런의 TfidfVectorizer를 TF-IDF 행렬을 만들 것입니다.  
텍스트 데이터에 있는 모든 단어를 가지고 행렬을 만들 수도 있겠지만, 여기서는 간단히 1,000개의 단어로 제한하겠습니다.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=1000) # 상휘 1000개의 단어 보존
x = vectorizer.fit_transform(text['headline_text'])
x.shape

(1082168, 1000)

1,082,168 × 1,000의 크기를 가진 가진 TF-IDF 행렬이 생겼습니다. 이제 이에 LDA를 수행합니다.

##### 4) 토픽 모델링

In [20]:
from sklearn.decomposition import LatentDirichletAllocation

ldamodel = LatentDirichletAllocation(n_components=10, learning_method='online', random_state = 777, max_iter=1)
lda_top=ldamodel.fit_transform(x)

print(ldamodel.components_)
print(ldamodel.components_.shape)

[[1.00001787e-01 1.00001305e-01 1.00004812e-01 ... 1.00006301e-01
  1.00004356e-01 1.00002683e-01]
 [1.00001851e-01 1.13508716e+03 1.00010774e-01 ... 1.00008428e-01
  1.00002315e-01 1.00002614e-01]
 [1.00001547e-01 1.00000897e-01 1.00004252e-01 ... 1.00004055e-01
  1.00001757e-01 7.53409554e+02]
 ...
 [1.00001119e-01 1.00001828e-01 1.00006994e-01 ... 1.00007152e-01
  1.00002998e-01 1.00004489e-01]
 [1.00002995e-01 1.00000806e-01 1.00004320e-01 ... 1.00003662e-01
  1.00001149e-01 1.00004879e-01]
 [1.00003429e-01 1.00002476e-01 1.00012101e-01 ... 1.00003613e-01
  1.00002605e-01 1.00006032e-01]]
(10, 1000)


In [25]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1)
              , [(feature_names[i], topic[i].round(2)) 
                 for i in topic.argsort()[:-n-1:-1]])
        
get_topics(ldamodel.components_, terms)      

Topic 1: [('government', 8724.47), ('sydney', 8387.49), ('queensland', 7720.29), ('change', 5874.83), ('home', 5670.93)]
Topic 2: [('australia', 13692.02), ('australian', 11084.92), ('melbourne', 7525.59), ('world', 6708.04), ('south', 6672.64)]
Topic 3: [('interview', 5924.98), ('kill', 5840.58), ('jail', 4627.45), ('life', 4274.88), ('health', 4267.65)]
Topic 4: [('house', 6113.7), ('canberra', 6111.84), ('2016', 5488.24), ('state', 4922.6), ('brisbane', 4856.3)]
Topic 5: [('court', 7541.27), ('attack', 6957.33), ('perth', 6456.74), ('open', 5660.24), ('face', 5194.16)]
Topic 6: [('plan', 6032.03), ('rural', 5503.3), ('death', 4999.48), ('indigenous', 4222.48), ('record', 3986.16)]
Topic 7: [('charge', 8425.83), ('election', 7562.16), ('adelaide', 6756.06), ('make', 5658.45), ('market', 5544.54)]
Topic 8: [('police', 12081.68), ('crash', 5277.09), ('drug', 4291.01), ('rise', 4044.02), ('beat', 3259.53)]
Topic 9: [('labor', 4047.75), ('national', 4038.68), ('council', 4006.73), ('leag